In [85]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from utils import *

# Soft voting

In [105]:
def mul(lst):
    return np.asarray(eval(lst))

def tolist(txt):
    return txt.tolist()

In [204]:
k = int(input()) # 앙상블 모델 개수 입력
s = nn.Softmax()

mod_pow = torch.tensor(list(map(float, input().split()))) # score 입력
mod_pow = s(mod_pow)

tmp = pd.DataFrame()
output = pd.DataFrame()
for i in range(k):
    path = input().strip() # score csv 파일 경로 입력
    data = pd.read_csv(path) 
    tmp[i] = data['probs'].apply(mul) * float(mod_pow[i])
output['id'] = data['id']
output['probs'] = np.sum(tmp, axis=1)
pred = output['probs'].apply(np.argmax).tolist()
pred_a = num_to_label(pred)
output["pred_label"] = pred_a
output['probs'] = output['probs'].apply(tolist)
output

/tmp/ipykernel_16950/3711341420.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  mod_pow = s(mod_pow)


,id,probs,pred_label
0,0,"[0.20558409500522448, 0.008355306859450871, 0....",org:product
1,1,"[0.03815876760101453, 0.0006037889250693184, 0...",per:alternate_names
2,2,"[0.9968576724088735, 6.785941651468863e-05, 0....",no_relation
3,3,"[0.917959742415383, 0.0011208676192146954, 0.0...",no_relation
4,4,"[0.9254844238311672, 0.059511495308409636, 0.0...",no_relation
...,...,...,...
7760,7760,"[0.18494721324059724, 0.0017169667227166744, 0...",org:place_of_headquarters
7761,7761,"[0.9924643221680528, 0.00045990009359553807, 0...",no_relation
7762,7762,"[0.0011133969121025409, 0.9967393451201745, 8....",org:top_members/employees
7763,7763,"[0.9003427814443764, 0.0007796729551131317, 0....",no_relation


In [205]:
output.to_csv('4e_submission.csv', index=False)

# Hard voting

In [147]:
roberta_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/result/roberta_ensemble_submission.csv')
electra_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/result/electra_ensemble_test_submission3.csv')
r_roberta_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/result/r_roberta_submission.csv')
electra_roberta_r_roberta_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/result/electra_roberta_r_roberta_submission.csv')
subm_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/subm.csv')
bigbird_df = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/bibbird_cr_cs_od_submission(83.46).csv')
bigbird_df2 = pd.read_csv('/opt/ml/code/level2_klue_nlp-level1-nlp-12/pl/ensemble/bigbird_ls_cs_od_submission(83.14).csv')

In [190]:
hard = pd.concat([roberta_df['pred_label'], 
                  electra_df['pred_label'],
                  r_roberta_df['pred_label'], 
                  electra_roberta_r_roberta_df['pred_label'],
                  subm_df['pred_label'], 
                  bigbird_df['pred_label'], 
                  bigbird_df['pred_label']], 
                 axis=1).mode(axis=1).iloc[:,0] # pred_label 최빈값

e_r_rr_s_bb = pd.concat([electra_roberta_r_roberta_df.iloc[:,:2],hard], axis=1) 
e_r_rr_s_bb.columns = ['id', 'probs', 'pred_label']
e_r_rr_s_bb

,id,probs,pred_label
0,0,"[0.20558580300150375, 0.008355345669877622, 0....",org:product
1,1,"[0.038144471842962845, 0.0006038049947596078, ...",per:alternate_names
2,2,"[0.9968577110318892, 6.785996457642538e-05, 0....",no_relation
3,3,"[0.9179619232446571, 0.0011209050470618012, 0....",no_relation
4,4,"[0.9255162641663264, 0.05947922989534496, 0.00...",no_relation
...,...,...,...
7760,7760,"[0.18494745634148418, 0.0017169836175552865, 0...",org:place_of_headquarters
7761,7761,"[0.9924641356023467, 0.00045991581719835155, 0...",no_relation
7762,7762,"[0.0011134248543043985, 0.9967392894376907, 8....",org:top_members/employees
7763,7763,"[0.9003420282817597, 0.0007797028477415893, 0....",no_relation


In [191]:
e_r_rr_s_bb.to_csv('e_r_rr_s_bb_hard_submission.csv', index=False)